In [18]:
import pandas as pd
# import json
import pickle
from pyomo.environ import *
from itertools import combinations

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# dem = pd.read_csv('C:/Jupyter/TESIS/instancias/demanda_n100_k30_t300.txt')
# preco = pd.read_csv('C:/Jupyter/TESIS/instancias/preco_n100_k30_t300.txt')

# dem = pd.read_excel('C:/tesis_wilmer/tesis/instancias/demanda_n7_k150_t60.xlsx')
# preco = pd.read_excel('C:/tesis_wilmer/tesis/instancias/preco_n7_k150_t60.xlsx')

#dem = pd.read_csv('C:/tesis_wilmer/tesis/instancias/demanda_n15_k200_t60.txt')
#preco = pd.read_csv('C:/tesis_wilmer/tesis/instancias/preco_n15_k200_t60.txt')

# dem_ini = pd.read_csv('C:/tesis_wilmer/tesis/dem_ini.txt')
# dem_anex = pd.read_csv('C:/tesis_wilmer/tesis/demanda_anex_t.txt')

# preco_ini = pd.read_csv('C:/tesis_wilmer/tesis/preco_ini.txt')
# preco_anex = pd.read_csv('C:/tesis_wilmer/tesis/preco_anex_n2.txt')

# dem = pd.concat([dem_ini, dem_anex], axis=0)
# preco = pd.read_csv('C:/tesis_wilmer/tesis/preco_ini.txt')
# preco = pd.concat([preco_ini, preco_anex], axis=0)

In [23]:
# dem_cor = dem.set_index(['origem', 'destino', 'classe','periodo'])['demanda'].to_dict()
# preco_cor = preco.set_index(['origem', 'destino', 'classe',])['preco'].to_dict()

# with open("dem_pro.json", "r") as fp:
#     dem_cor = json.load(fp)

# with open("preco_pro.json", "r") as fp:
#     preco_cor = json.load(fp)


with open("preco_pro1.pkl", 'rb') as archivo:
    preco_cor = pickle.load(archivo)

with open("dem_pro1.pkl", 'rb') as archivo:
    dem_cor = pickle.load(archivo)


In [24]:
cap = 100 #capacidad del tren
n = 5 #numero de estaciones
kk = 10 #numero de classes
tt = 10 #numero de periodos

In [25]:
estacoes = list(range(n+1))
combinaciones = list(combinations(estacoes, 2))
oridest = [(inicio, fin) for inicio, fin in combinaciones if estacoes.index(inicio) < estacoes.index(fin)]

In [27]:
# instanciar o modelo
model = AbstractModel()

# Escalares
model.n = Param(initialize = n)
model.i = Param(initialize = n)
model.j = Param(initialize = n)
model.k = Param(initialize = kk)
model.t = Param(initialize = tt)
model.Q = Param(initialize = cap)

# conjuntos
model.I = RangeSet(1, model.i)
model.J = RangeSet(1, model.j)
model.K = RangeSet(1, model.k)
model.T = RangeSet(1, model.t)

#parâmetros indexados
model.P = Param(model.I, model.J, model.K, initialize = preco_cor)
model.d = Param(model.I, model.J, model.K, model.T, initialize = dem_cor)

# variáveis de decisão
model.X = Var(RangeSet(0, model.i), model.J, model.K, model.T, within = NonNegativeIntegers)
model.Y = Var(RangeSet(0, model.i), model.J, model.K, model.T, within = NonNegativeIntegers)
model.A = Var(RangeSet(0, model.i), within = NonNegativeIntegers)

# função objetivo
def fo1(model):
    return sum(model.P[i,j,k]*(model.X[i,j,k,t]) for i in model.I for j in model.J for k in model.K for t in model.T if i<j)
model.OBJ1 = Objective(rule = fo1, sense = maximize)

# restrições
def cap1(model, i):
    return model.A[i] ==  model.A[i-1] - sum((model.X[i-1,j,k,t]) for j in model.J for k in model.K for t in model.T if j>=i) + sum((model.X[j,i,k,t]) for j in model.J for k in model.K for t in model.T if j<i)
model.ConstrainCap1 = Constraint(model.I, rule = cap1)

def cap2(model, i):
    if i < model.n:
        return sum((model.X[i,j,k,t]) for j in model.J for k in model.K for t in model.T if i<j) <= model.A[i]
    else:
        return Constraint.Skip
model.ConstrainCap2 = Constraint(model.I, rule = cap2)

def authorization(model, i, j, k, t):
    if i < j:
        return model.Y[i,j,k,t] >= model.X[i,j,k,t]
    else:
        return Constraint.Skip
model.ConstrainAuthorization = Constraint(model.I, model.J, model.K, model.T, rule = authorization)

def classe(model, i, j, k, t):
    if i < j and k < model.k:
        return model.Y[i,j,k,t] <= model.Y[i,j,k+1,t]
    else:
        return Constraint.Skip
model.ConstrainClasse = Constraint(model.I, model.J, model.K, model.T, rule = classe)

def assignment(model, i, j, k, t):
    if i < j:
        return model.X[i,j,k,t] <= model.d[i,j,k,t]
    else:
        return Constraint.Skip
model.ConstrainAssignment = Constraint(model.I, model.J, model.K, model.T,rule = assignment)

def assignment0(model, j, k, t):
    return model.X[0,j,k, t] == 0
model.ConstrainAssignment0 = Constraint(model.J, model.K, model.T, rule = assignment0)

def authorization0(model, j, k, t):
    return model.Y[0,j,k,t] == 0
model.ConstrainAuthorization0 = Constraint(model.J, model.K, model.T, rule = authorization0)

def cap0(model):
    return model.A[0] == model.Q
model.ConstrainCap0 = Constraint(rule = cap0)

instance = model.create_instance()
# instance.pprint()

opt = SolverFactory('cplex')
results = opt.solve(instance, tee=True)
# print(results)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Wilmer\AppData\Local\Temp\tmpdca4r8na.cplex.log' open.
CPLEX> Problem 'C:\Users\Wilmer\AppData\Local\Temp\tmp1nvhd_a_.pyomo.lp' read.
Read time = 0.09 sec. (0.32 ticks)
CPLEX> Problem name         : C:\Users\Wilmer\AppData\Local\Temp\tmp1nvhd_a_.pyomo.lp
Objective sense      : Maximize
Variables            :    3006  [General Integer: 3006]
Objective nonzeros   :    1000
Linear constraints   :    3910  [Less: 2904,  Equal: 1006]
  Nonzeros           :    8515
  RHS nonzeros       :    1001

Variables            : Min LB: 0.000000         Max UB: all infinite   
Objective nonzeros   : 

In [ ]:
print('Valor da função objetivo: ', '\033[1m' + '\033[32m' + str(value(instance.OBJ1)) + '\033[0m')

print('')

# print('Valores das variáveis de decisão: ')
lista = []
for i in range(1,n+1):
    for j in range(1,n+1):
        for k in range(1,kk+1):
            for t in range(1,tt+1):
                if i<j and value(instance.Y[i,j,k,t])!=0:
                    # print(i,j,k, ' - ',value(instance.Y[i,j,k]), ' - ', value(instance.X[i,j,k]) )
                    lista.append([str(i) + '-' + str(j), k, t, value(instance.P[i,j,k]), value(instance.d[i,j,k,t]) , value(instance.X[i,j,k,t]), value(instance.Y[i,j,k,t]) ])

a = pd.DataFrame(lista, columns=['o-d','classe','Periodo','preco','demanda','Assignments','Authorizations'])
print(a)

Valor da função objetivo:  8337000.0

    o-d  classe  Periodo  preco  demanda  Assignments  Authorizations
0   1-2     150        1   1978       23         23.0            23.0
1   1-2     150        2   1978       33         33.0            33.0
2   1-2     150        3   1978       16         16.0            16.0
3   1-2     150        4   1978        9          9.0             9.0
4   1-2     150        5   1978       94         94.0            94.0
5   1-2     150        6   1978       19         19.0            19.0
6   1-2     150        7   1978       44         44.0            44.0
7   1-2     150        8   1978       68         68.0            68.0
8   1-2     150        9   1978       55         55.0            55.0
9   1-2     150       10   1978       59         59.0            59.0
10  1-2     150       11   1978       46         46.0            46.0
11  1-2     150       12   1978       26         26.0            26.0
12  1-2     150       13   1978       10         10.

In [ ]:
# print('')
# print('Valores das capacidades em cada nó: ')

# for i in range(1, n+1):
#     print(i , ' - ', value(instance.A[i]))

# print('')
# tabela_pivote = pd.pivot_table(a[a['classe']==1],  values='Authorizations', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')
# tabela_pivote2 = pd.pivot_table(a[a['classe']==1],  values='Assignments', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')

# tabela_pivote3 = pd.pivot_table(a[a['classe']==2],  values='Authorizations', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')
# tabela_pivote4 = pd.pivot_table(a[a['classe']==2],  values='Assignments', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')

# result_1 = tabela_pivote + tabela_pivote2
# result_2 = tabela_pivote3 + tabela_pivote4

# print(result_1, '\n \n', result_2)

# a = pd.DataFrame(lista, columns=['o-d','classe','Periodo','preco','demanda','Assignments','Authorizations'])
# a.to_excel('modelo_1_1.xlsx', index=False)

# corregit instancias
# dem_cor_1 = {(i,j,k,t): dem_cor[(i,j,k,t)] if (i,j,k,t) in dem_cor else 0  for i in range(1, n+1) for j in range(1, n+1) for k in range(1, kk+1) for t in range(1, tt+1)}
# preco_cor_1 = {(i,j,k): preco_cor[(i,j,k)] if (i,j,k) in preco_cor else 0  for i in range(1, n+1) for j in range(1, n+1) for k in range(1, kk+1)}